We have data of 7 markets

Fields: Datetime, bid, ask

Bid price: Aggregate price at which sellers want to sell. I'll sell at Bid price.
Ask price: Aggregate price at which buyers want to buy. I'll buy at Ask price.
Spread: Difference between Bid price and ask price

Arbitrage theory: As long as there is spread you make money, bigger the spread bigger the return. 
Doesnt exist anymore except in hard to track markets, because of visibility and market efficiency.

Solution:

Aggregate all markets in one dataframe based on timestamp.

At that timestamp find the max spread, go all in hoping that there is buyer and seller volume, since that data is not given. 

In [16]:
pip install glob

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
import pandas as pd
import numpy as np
import os

In [41]:
# Get a list of all the CSV files in the specified directory
csv_files = glob.glob('Arbitrage-Algo-dataset/datasets/*.csv')

print(csv_files)

['Arbitrage-Algo-dataset/datasets\\market-1.csv', 'Arbitrage-Algo-dataset/datasets\\market-2.csv', 'Arbitrage-Algo-dataset/datasets\\market-3.csv', 'Arbitrage-Algo-dataset/datasets\\market-4.csv', 'Arbitrage-Algo-dataset/datasets\\market-5.csv', 'Arbitrage-Algo-dataset/datasets\\market-6.csv', 'Arbitrage-Algo-dataset/datasets\\market-7.csv']


In [55]:
# Dictionary to store dataframes
dfs = {}

# Read each CSV file into a dataframe and store in the dictionary
for csv_file in csv_files:
    df = pd.read_csv(csv_file, sep=';')
    # print(csv_file[-12:-4]+"-bid")
    df = df.rename(columns={'bid':csv_file[-12:-4]+"-bid", 'ask':csv_file[-12:-4]+"-ask"})
    print(df.head())
    dfs[csv_file] = df


# Now, dfs is a dictionary where keys are filenames and values are dataframes
# You can access a specific dataframe using dfs['filename.csv']
# For example, print the first few rows of the first dataframe:
print(dfs[csv_files[0]].head())

              Datetime  market-1-bid  market-1-ask
0  2017-04-27 19:58:12        1428.1        1428.2
1  2017-04-27 19:58:15        1428.1        1428.2
2  2017-04-27 19:58:18        1428.2        1428.9
3  2017-04-27 19:58:21        1427.5        1428.9
4  2017-04-27 19:58:24        1427.5        1428.9
              Datetime  market-2-bid  market-2-ask
0  2017-04-27 19:58:12       1326.08       1327.34
1  2017-04-27 19:58:15       1326.08       1327.34
2  2017-04-27 19:58:18       1326.09       1327.34
3  2017-04-27 19:58:21       1326.09       1327.34
4  2017-04-27 19:58:24       1326.11       1327.33
              Datetime  market-3-bid  market-3-ask
0  2017-04-27 19:58:12       1298.84        1299.5
1  2017-04-27 19:58:15       1298.84        1299.5
2  2017-04-27 19:58:18       1298.39        1299.5
3  2017-04-27 19:58:21       1298.39        1299.5
4  2017-04-27 19:58:24       1298.39        1299.5
              Datetime  market-4-bid  market-4-ask
0  2017-04-27 19:58:12        1

In [56]:
dfs[csv_files[0]]

,Datetime,market-1-bid,market-1-ask
0,2017-04-27 19:58:12,1428.1,1428.2
1,2017-04-27 19:58:15,1428.1,1428.2
2,2017-04-27 19:58:18,1428.2,1428.9
3,2017-04-27 19:58:21,1427.5,1428.9
4,2017-04-27 19:58:24,1427.5,1428.9
...,...,...,...
534527,2017-05-28 14:03:00,2139.8,2143.0
534528,2017-05-28 14:03:03,2139.8,2143.0
534529,2017-05-28 14:03:06,2140.9,2143.0
534530,2017-05-28 14:03:09,2140.9,2143.0


In [74]:
from functools import reduce

# Extract all dataframes from the dictionary
dataframes_list = list(dfs.values())

# Use reduce to iteratively merge the dataframes
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Datetime', how='outer'), dataframes_list)

# Print the first few rows of the merged dataframe
print(merged_df.head())


              Datetime  market-1-bid  market-1-ask  market-2-bid  \
0  2017-04-27 19:58:12        1428.1        1428.2       1326.08   
1  2017-04-27 19:58:15        1428.1        1428.2       1326.08   
2  2017-04-27 19:58:18        1428.2        1428.9       1326.09   
3  2017-04-27 19:58:21        1427.5        1428.9       1326.09   
4  2017-04-27 19:58:24        1427.5        1428.9       1326.11   

   market-2-ask  market-3-bid  market-3-ask  market-4-bid  market-4-ask  \
0       1327.34       1298.84        1299.5        1339.8       1339.98   
1       1327.34       1298.84        1299.5        1339.8       1339.98   
2       1327.34       1298.39        1299.5        1339.8       1339.98   
3       1327.34       1298.39        1299.5        1339.8       1339.98   
4       1327.33       1298.39        1299.5        1339.8       1339.98   

   market-5-bid  market-5-ask  market-6-bid  market-6-ask  market-7-bid  \
0       1324.88       1328.99       1288.03       1290.75        

In [75]:
merged_df.to_csv(r'C:\Users\LENOVO PRO\Workspace\Projects\Data-Science-Projects\Arbitrage-Algo.csv', index=False)


In [73]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datetime             659 non-null    object 
 1   market-1-bid         659 non-null    float64
 2   market-1-ask         659 non-null    float64
 3   market-2-bid         659 non-null    float64
 4   market-2-ask         659 non-null    float64
 5   market-3-bid         659 non-null    float64
 6   market-3-ask         659 non-null    float64
 7   market-4-bid         659 non-null    float64
 8   market-4-ask         659 non-null    float64
 9   market-5-bid         659 non-null    float64
 10  market-5-ask         659 non-null    float64
 11  market-6-bid         659 non-null    float64
 12  market-6-ask         659 non-null    float64
 13  market-7-bid         659 non-null    float64
 14  market-7-ask         659 non-null    float64
 15  ask-price            659 non-null    flo

In [69]:
# Extract the columns related to ask prices
ask_columns = [col for col in merged_df.columns if col.endswith('-ask')]

# Create a new column 'ask-price' with the minimum ask price for each row
merged_df['ask-price'] = merged_df[ask_columns].min(axis=1)

# Extract the columns related to bid prices
bid_columns = [col for col in merged_df.columns if col.endswith('-bid')]

# Create a new column 'bid-price' with the minimum bid price for each row
merged_df['bid-price'] = merged_df[bid_columns].max(axis=1)

# Create columns for bid-market and ask-market
merged_df['bid-market'] = merged_df.filter(like='-bid').idxmax(axis=1).apply(lambda x: x.split('-')[1])
merged_df['ask-market'] = merged_df.filter(like='-ask').idxmin(axis=1).apply(lambda x: x.split('-')[1])

# Create a new column 'spread' 
merged_df['spread'] = merged_df['bid-price'] - merged_df['ask-price']


# Assuming merged_df is your dataframe
markets = [f'market-{i}' for i in range(1, 8)]

# Create a random volume between 1 and 50 for each market
for market in markets:
    merged_df[f'{market}-Bid-volume'] = np.random.randint(1, 51, size=len(merged_df))
    merged_df[f'{market}-Ask-volume'] = np.random.randint(1, 51, size=len(merged_df))


# Print the updated dataframe
print(merged_df.head())


              Datetime  market-1-bid  market-1-ask  market-2-bid  \
0  2017-04-27 19:58:12        1428.1        1428.2       1326.08   
1  2017-04-27 19:58:15        1428.1        1428.2       1326.08   
2  2017-04-27 19:58:18        1428.2        1428.9       1326.09   
3  2017-04-27 19:58:21        1427.5        1428.9       1326.09   
4  2017-04-27 19:58:24        1427.5        1428.9       1326.11   

   market-2-ask  market-3-bid  market-3-ask  market-4-bid  market-4-ask  \
0       1327.34       1298.84        1299.5        1339.8       1339.98   
1       1327.34       1298.84        1299.5        1339.8       1339.98   
2       1327.34       1298.39        1299.5        1339.8       1339.98   
3       1327.34       1298.39        1299.5        1339.8       1339.98   
4       1327.33       1298.39        1299.5        1339.8       1339.98   

   market-5-bid  ...  market-3-Bid-volume  market-3-Ask-volume  \
0       1324.88  ...                   21                   22   
1       

In [ ]:
Investment = 1000

Demand = rand